In [18]:
#!pip install tiktoken

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from dotenv import load_dotenv

from langchain.callbacks import get_openai_callback
import os
import sys
import json
import tiktoken
load_dotenv()


path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils


### Função para contar tokens

In [20]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def utf8len(s):
    return len(s.encode('utf-8'))

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)


In [21]:
data = {'basic_prompt':{}, 'openai_demonstration_prompt':{}, 'code_representation_prompt':{}}

## Mondial

In [ ]:

SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)
db.get_table_names()

### Basic Prompt

In [ ]:
data['basic_prompt']['mondial_size'] = utf8len(db_mondial.get_schema_basic_prompt().strip())
data['basic_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_schema_basic_prompt().strip())
print(data['basic_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_schema_basic_prompt().strip()), "tokens")

In [24]:
print(db_mondial.get_schema_basic_prompt())

Table continent, columns = [*,name,area]
Table country, columns = [*,name,code,capital,province,area,population]
Table desert, columns = [*,name,area]
Table island, columns = [*,name,islands,area,elevation,type]
Table mountain, columns = [*,name,mountains,elevation,type]
Table sea, columns = [*,name,area,depth]
Table continent, columns = [*,name,area]
Table country, columns = [*,name,code,capital,province,area,population]
Table desert, columns = [*,name,area]
Table island, columns = [*,name,islands,area,elevation,type]
Table lake, columns = [*,name,river,area,elevation,depth,height,type]
Table mountain, columns = [*,name,mountains,elevation,type]
Table river, columns = [*,name,river,lake,sea,length,area,mountains,sourceelevation,estuaryelevation]
Table sea, columns = [*,name,area,depth]
Table borders, columns = [*,country1,country2,length]
Table countrylocalname, columns = [*,country,localname]
Table countryothername, columns = [*,country,othername]
Table countrypops, columns = [*,coun

### OpenAI Demonstration Prompt

In [25]:
data['openai_demonstration_prompt']['mondial_size'] = utf8len(db_mondial.get_schema_openai_prompt().strip())
data['openai_demonstration_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_schema_openai_prompt().strip())
print(data['openai_demonstration_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_schema_openai_prompt().strip()), "tokens")

726
726 tokens


In [26]:
print(db_mondial.get_schema_openai_prompt())

# continent (name, area)
# country (name, code, capital, province, area, population)
# desert (name, area)
# island (name, islands, area, elevation, type)
# mountain (name, mountains, elevation, type)
# sea (name, area, depth)
# continent (name, area)
# country (name, code, capital, province, area, population)
# desert (name, area)
# island (name, islands, area, elevation, type)
# lake (name, river, area, elevation, depth, height, type)
# mountain (name, mountains, elevation, type)
# river (name, river, lake, sea, length, area, mountains, sourceelevation, estuaryelevation)
# sea (name, area, depth)
# borders (country1, country2, length)
# countrylocalname (country, localname)
# countryothername (country, othername)
# countrypops (country, year, population)
# economy (country, gdp, agriculture, service, industry, inflation, unemployment)
# encompasses (country, continent, percentage)
# ethnicgroup (country, name, percentage)
# islandin (island, sea, lake, river)
# lake (name, river, are

### Code Representation Prompt

In [ ]:
data['code_representation_prompt']['mondial_size'] = utf8len(db_mondial.get_table_info().strip())
data['code_representation_prompt']['mondial_tokens_usage'] = num_tokens_from_string(db_mondial.get_table_info().strip())
print(data['code_representation_prompt']['mondial_tokens_usage'])
print(llm.get_num_tokens(db_mondial.get_table_info().strip()), "tokens")

In [28]:
print(db_mondial.get_table_info())


CREATE TABLE "MONDIAL_GPT".airport (
	iatacode VARCHAR(3 CHAR) NOT NULL, 
	name VARCHAR(100 CHAR), 
	country VARCHAR(4 CHAR), 
	city VARCHAR(50 CHAR), 
	province VARCHAR(50 CHAR), 
	island VARCHAR(50 CHAR), 
	latitude NUMBER, 
	longitude NUMBER, 
	elevation NUMBER, 
	gmtoffset NUMBER, 
	CONSTRAINT sys_c00163097 PRIMARY KEY (iatacode), 
	CONSTRAINT airport_city_province_fk FOREIGN KEY(city, country, province) REFERENCES mondial_gpt.city (name, country, province), 
	CONSTRAINT airport_country_fk FOREIGN KEY(country) REFERENCES mondial_gpt.country (code), 
	CONSTRAINT airport_island_fk FOREIGN KEY(island) REFERENCES mondial_gpt.island (name), 
	CONSTRAINT airport_province_country_fk FOREIGN KEY(province, country) REFERENCES mondial_gpt.province (name, country), 
	CONSTRAINT airplat CHECK ((Latitude >= -90) AND (Latitude <= 90)), 
	CONSTRAINT airplon CHECK ((Longitude >= -180) AND (Longitude <= 180))
)


CREATE TABLE "MONDIAL_GPT".borders (
	country1 VARCHAR(4 CHAR) NOT NULL, 
	country2 V

### Basic Prompt

### Resultados

In [29]:
data

{'basic_prompt': {'mondial_size': 3586, 'mondial_tokens_usage': 953},
 'openai_demonstration_prompt': {'mondial_size': 2795,
  'mondial_tokens_usage': 726},
 'code_representation_prompt': {'mondial_size': 30269,
  'mondial_tokens_usage': 7920}}

In [30]:
import pandas as pd

df = pd.DataFrame(data)

In [31]:

df 

,basic_prompt,openai_demonstration_prompt,code_representation_prompt
mondial_size,3586,2795,30269
mondial_tokens_usage,953,726,7920


In [32]:
df.index
df.rename(index={'mondial_size': 'Mondial - Size (bytes)', 'mondial_tokens_usage':'Mondial - Tokens Usage'
                 }, inplace=True)

In [33]:
df

,basic_prompt,openai_demonstration_prompt,code_representation_prompt
Mondial - Size (bytes),3586,2795,30269
Mondial - Tokens Usage,953,726,7920


In [34]:
filename = 'prompt_representation_size.csv'
dir = 'results/'
df.to_csv(dir+filename)